### Some trajectory visualization code

In [ ]:
import math
import time
import krpc
import numpy as np
from collections import namedtuple

In [ ]:
conn = krpc.connect(name='Main')
KSC = conn.space_center
vessel = KSC.active_vessel
print(f"Vessel name: -- {vessel.name} -- is online\nStatus: -- {vessel.situation} --")

#### Launch
* Launch script from kRPC docs

In [ ]:
ut = conn.add_stream(getattr, conn.space_center, 'ut')
altitude = conn.add_stream(getattr, vessel.flight(), 'mean_altitude')
apoapsis = conn.add_stream(getattr, vessel.orbit, 'apoapsis_altitude')
tgt_roll = vessel.auto_pilot.target_roll
turn_start_altitude = 250
turn_end_altitude = 45000
target_altitude = 150000
vessel.control.sas = False
vessel.control.rcs = False
vessel.control.throttle = 1.0

In [ ]:
# Activate the first stage
vessel.control.activate_next_stage()
vessel.auto_pilot.engage()
vessel.auto_pilot.target_pitch_and_heading(90, 90)

In [ ]:
# Main ascent loop
srbs_separated = False
turn_angle = 0
while True:
    # Gravity turn
    if altitude() > turn_start_altitude and altitude() < turn_end_altitude:
        frac = ((altitude() - turn_start_altitude) /
                (turn_end_altitude - turn_start_altitude))
        new_turn_angle = frac * 90
        if abs(new_turn_angle - turn_angle) > 0.5:
            turn_angle = new_turn_angle
            vessel.auto_pilot.target_pitch_and_heading(90-turn_angle, 90)

### Test
* Drawing some reference vectors for visualization

In [ ]:
vessel_direction = vessel.direction(vessel.surface_reference_frame) 
horizon_direction = (0,vessel_direction[1],vessel_direction[2])
conn.drawing.add_direction((1,0,0),vessel.reference_frame,length=100)
conn.drawing.add_direction((0,1,0),vessel.reference_frame,length=100)
conn.drawing.add_direction(horizon_direction,vessel.surface_reference_frame,length=100)
conn.drawing.add_direction((0,0,1),vessel.orbit.body.reference_frame,length=100)
time.sleep(30)
conn.drawing.clear()